In [9]:
# !pip uninstall tensorflow keras 

In [10]:
# !pip install tensorflow keras keras-cv-attention-models glob2 pandas tqdm scikit-learn scikit-image

In [11]:
# !pip install tensorflow==2.7.0 keras==2.7 keras-cv-attention-models glob2 pandas tqdm scikit-learn scikit-image

In [12]:
# !pip install tensorflow keras

In [13]:
# !pip install keras-cv-attention-models

In [1]:
import os
from tensorflow import keras
import losses, train, GhostFaceNets
import tensorflow as tf
import keras_cv_attention_models

In [2]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# Remove the below for better accuracies and keep it for faster training
keras.mixed_precision.set_global_policy("mixed_float16")

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4060 Ti, compute capability 8.9


GhostFaceNetV1

In [4]:
# ms1m-retinaface-t1 (MS1MV3) dataset
# data_basic_path = 'datasets/ms1m-retinaface-t1'
# data_path = data_basic_path + '_112x112_folders'
# eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'datasets/data_0_200_112x112_folders'
eval_paths = ['datasets/data_0_200/lfw.bin', 'datasets/data_0_200/cfp_fp.bin', 'datasets/data_0_200/agedb_30.bin']

In [5]:
#GhostFaceNetV1
# # Strides of 2
# basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
# basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
# basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

>>>> Change BatchNormalization momentum and epsilon default value.
>>>> Convert ReLU: activation --> activation
>>>> Convert ReLU: activation_1 --> activation_1
>>>> Convert ReLU: activation_2 --> activation_2
>>>> Convert ReLU: activation_3 --> activation_3
>>>> Convert ReLU: activation_4 --> activation_4
>>>> Convert ReLU: activation_5 --> activation_5
>>>> Convert ReLU: activation_6 --> activation_6
>>>> Convert ReLU: activation_7 --> activation_7
>>>> Convert ReLU: activation_8 --> activation_8
>>>> Convert ReLU: activation_9 --> activation_9
>>>> Convert ReLU: activation_11 --> activation_11
>>>> Convert ReLU: activation_12 --> activation_12
>>>> Convert ReLU: activation_13 --> activation_13
>>>> Convert ReLU: activation_15 --> activation_15
>>>> Convert ReLU: activation_16 --> activation_16
>>>> Convert ReLU: activation_17 --> activation_17
>>>> Convert ReLU: activation_18 --> activation_18
>>>> Convert ReLU: activation_19 --> activation_19
>>>> Convert ReLU: activation_20 --> ac

In [6]:
# # Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s2_new_data_0_200.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

# # Strides of 1
# tt = train.Train(data_path, eval_paths=eval_paths,
#     save_path='ghostnetv1_w1.3_s1_new_data_0_200.h5',
#     basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
#     batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

>>>> L2 regularizer value from basic_model: 0.00025


In [7]:
print(dir(tt))  # Kiểm tra xem 'train' có nằm trong danh sách không


['__add_emb_output_to_model__', '__basic_train__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_dataset__', '__init_emb_losses__', '__init_model__', '__init_optimizer__', '__init_subclass__', '__init_type_by_loss__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__search_embedding_layer__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'arcface', 'arcface_partial', 'basic_model', 'batch_size', 'batch_size_per_replica', 'center', 'classes', 'custom_callbacks', 'data_path', 'default_optimizer', 'distill', 'distill_emb_map_layer', 'gently_stop', 'image_per_class', 'inited_from_model', 'is_distill_ds', 'is_triplet_dataset', 'lr_scheduler', 'metrics', 'mixup_alpha', 'model', 'model_checkpoint', 'my_evals', 'my_history', 'output_weight_decay', 'partial_fc_split', 'pretrained', 'random_cutout_mask_are

In [8]:

optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
# tt.steps_per_epoch = 10
tt.train(sch, 0)

>>>> Init type by loss function name...
>>>> Train arcface...
>>>> Init softmax dataset...
>>>> reloaded from dataset backup: data_0_200_112x112_folders_shuffle.npz
>>>> Image length: 14472, Image class length: 14472, classes: 201
>>>> Use specified optimizer: <keras.optimizer_v2.gradient_descent.SGD object at 0x000001D2CEFBAEE0>
>>>> Add L2 regularizer to model output layer, output_weight_decay = 0.000500
>>>> Add arcface layer, arc_kwargs={'loss_top_k': 1, 'append_norm': False, 'partial_fc_split': 0, 'name': 'arcface'}, vpl_kwargs={'vpl_lambda': 0.15, 'start_iters': -113, 'allowed_delta': 200}...
>>>> loss_weights: {'arcface': 1}

Learning rate for iter 1 is 0.10000000149011612, global_iterNum is 0


KeyboardInterrupt: 

In [18]:
import h5py
print(h5py.__version__)


3.12.1


In [20]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Create a checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='D:/CODE/PYTHON/Do_an/code_anh_tien/GhostFaceNets/checkpointsbest_model.h5',  # Specify the path where the best model should be saved
    monitor='val_loss',  # Metric to monitor
    save_best_only=True,  # Save only when the monitored metric improves
    mode='min'  # The mode can be 'min', 'max', or 'auto'
)

# Add this callback to the list of callbacks in the Train class
tt.callbacks.append(checkpoint_callback)


In [27]:
!pip uninstall h5py

^C


In [26]:
import tensorflow as tf
print(tf.__version__)
import h5py
print(h5py.__version__)


2.7.0
3.12.1


In [1]:
model_save_path = 'your_model.h5'  # Specify the desired file path with .h5 extension
tt.model.save(model_save_path)


NameError: name 'tt' is not defined

In [ ]:
# import os
# from tensorflow import keras

# def save_model(model, save_path):
#     if model is None:
#         print(">>>> Error: Model is not initialized.")
#         return
    
#     # Save the model to the specified path
#     try:
#         model.save(save_path)
#         print(f">>>> Model saved successfully to {save_path}")
#     except Exception as e:
#         print(f">>>> Error in saving model: {str(e)}")

# if __name__ == "__main__":
#     # Example load and save
#     try:
#         # Assuming the model is loaded from the checkpoint
#         model = keras.models.load_model("your_model_checkpoint.h5")
#         save_model(model, "new_saved_model.h5")
#     except Exception as e:
#         print(f">>>> Error in loading or saving model: {str(e)}")


In [11]:
import os
from tensorflow import keras

def save_basic_model(basic_model, save_path):
    latest_save_path = os.path.join("checkpoints", os.path.splitext(save_path)[0] + "_basic_model_latest.h5")
    print(">>>> Saving latest basic model to:", latest_save_path)
    basic_model.save(latest_save_path)
    print(">>>> Model saved successfully!")


In [ ]:
# !pip install h5py


GhostFaceNetV2

In [7]:
# # ms1m-retinaface-t1 (MS1MV3) dataset
# data_basic_path = 'datasets/ms1m-retinaface-t1'
# data_path = data_basic_path + '_112x112_folders'
# eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'datasets/faces_emore_112x112_folders'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [8]:
# #GhostFaceNetV2
# # Strides of 2
# basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
# basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
# basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

>>>> Change BatchNormalization momentum and epsilon default value.
>>>> Convert ReLU: stack4_se_relu --> stack4_se_prelu
>>>> Convert ReLU: stack5_se_relu --> stack5_se_prelu
>>>> Convert ReLU: stack10_se_relu --> stack10_se_prelu
>>>> Convert ReLU: stack11_se_relu --> stack11_se_prelu
>>>> Convert ReLU: stack12_se_relu --> stack12_se_prelu
>>>> Convert ReLU: stack14_se_relu --> stack14_se_prelu
>>>> Convert ReLU: stack16_se_relu --> stack16_se_prelu


TypeError: backbones.ghostv2.GhostNetV2() got multiple values for keyword argument 'stem_strides'

In [ ]:
# #Strides of 2
# tt = train.Train(data_path, eval_paths=eval_paths,
#     save_path='ghostnetv2_w1.3_s2.h5',
#     basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
#     batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

CosFaceLoss

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.CosFaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.CosFaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

Subcenter ArcFace Loss

In [ ]:
# ms1m-retinaface-t1 (MS1MV3) dataset
data_basic_path = 'datasets/ms1m-retinaface-t1'
data_path = data_basic_path + '_112x112_folders'
eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'datasets/faces_emore_112x112_folders'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [ ]:
# GhostFaceNetV1
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

In [ ]:
# GhostFaceNetV2
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

In [ ]:
#GhostFaceNetV1
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s2_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s1_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
#GhostFaceNetV2
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s2_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s1_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
""" First, Train with `lossTopK = 3` """
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer, "lossTopK": 3},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50, "lossTopK": 3},
]
tt.train(sch, 0)

In [ ]:
""" Then drop non-dominant subcenters and high-confident noisy data, which is `>75 degrees` """
import data_drop_top_k
# data_drop_top_k.data_drop_top_k('./checkpoints/TT_mobilenet_topk_bs256.h5', '/datasets/faces_casia_112x112_folders/', limit=20)
new_data_path = data_drop_top_k.data_drop_top_k(tt.model, tt.data_path)

In [ ]:
""" Train with the new dataset again, this time `lossTopK = 1` """
tt.reset_dataset(new_data_path)

At this point you can continue training the model or start training again and considering the generated dataset as a new dataset

In [ ]:
# Continue training
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

In [ ]:
# Train the models from scatch using the new generated dataset

# New Cleaned Dataset
data_path = 'Path_To_New_Generated_Dataset'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [ ]:
# GhostFaceNetV1
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

In [ ]:
# GhostFaceNetV2
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

In [ ]:
# GhostFaceNetV1
# Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

# Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
# GhostFaceNetV2
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)